In [1]:
from trackml.dataset import load_event
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

In [2]:
hits, cells, particles, truth = load_event('./train_100_events/event000001000')

In [3]:
hits.head()

,hit_id,x,y,z,volume_id,layer_id,module_id
0,1,-64.409897,-7.163700,-1502.5,7,2,1
1,2,-55.336102,0.635342,-1502.5,7,2,1
2,3,-83.830498,-1.143010,-1502.5,7,2,1
3,4,-96.109100,-8.241030,-1502.5,7,2,1
4,5,-62.673599,-9.371200,-1502.5,7,2,1


In [4]:
particles.head()

,particle_id,vx,vy,vz,px,py,pz,q,nhits
0,4503668346847232,-0.009288,0.009861,-0.077879,-0.055269,0.323272,-0.203492,-1,8
1,4503737066323968,-0.009288,0.009861,-0.077879,-0.948125,0.470892,2.010060,1,11
2,4503805785800704,-0.009288,0.009861,-0.077879,-0.886484,0.105749,0.683881,-1,0
3,4503874505277440,-0.009288,0.009861,-0.077879,0.257539,-0.676718,0.991616,1,12
4,4503943224754176,-0.009288,0.009861,-0.077879,16.439400,-15.548900,-39.824902,1,3


In [5]:
truth.head()

,hit_id,particle_id,tx,ty,tz,tpx,tpy,tpz,weight
0,1,0,-64.411598,-7.164120,-1502.5,250710.000000,-149908.000000,-956385.00000,0.000000
1,2,22525763437723648,-55.338501,0.630805,-1502.5,-0.570605,0.028390,-15.49220,0.000010
2,3,0,-83.828003,-1.145580,-1502.5,626295.000000,-169767.000000,-760877.00000,0.000000
3,4,297237712845406208,-96.122902,-8.230360,-1502.5,-0.225235,-0.050968,-3.70232,0.000008
4,5,418835796137607168,-62.659401,-9.375040,-1502.5,-0.281806,-0.023487,-6.57318,0.000009


In [6]:
d = defaultdict(list)
for particle_id in range(len(truth)):
    key = truth.particle_id[particle_id]

    val = truth.hit_id[particle_id]
    d[key].append(val)

In [7]:
d2 = defaultdict(int)
for particle_id, hit_ids in d.items():
    p_hits = hits.iloc[np.array(hit_ids) - 1]
    diffs = p_hits[['x','y','z']].diff().reset_index().iloc[1:]
    arclength = np.sum(np.sqrt(np.sum(np.square(diffs), 1)))
    d2[particle_id] = arclength

In [27]:
d3 = defaultdict(float)
for p_id, hit_ids in d.items():
    if p_id != 0:
        row = particles[particles.particle_id == p_id]
        px,py,pz = row['px'],row['py'],row['pz']

           particle_id        vx        vy       vz       px        py  \
520  22525763437723648 -0.015802  0.006381  1.16279 -0.56967 -0.011187   

         pz  q  nhits  
520 -15.496  1     10  
             particle_id        vx        vy       vz        px        py  \
4112  297237712845406208 -0.000486 -0.015051  5.75865 -0.240629  0.012071   

           pz  q  nhits  
4112 -3.70766 -1     11  
             particle_id        vx        vy       vz        px        py  \
5567  418835796137607168  0.018366 -0.016865  4.19268 -0.268943 -0.058487   

           pz  q  nhits  
5567 -6.58619  1     10  
             particle_id        vx        vy       vz        px       py  \
1604  108087696726949888  0.010383 -0.012398  3.92894 -0.392669 -0.07547   

           pz  q  nhits  
1604 -10.4714  1     11  
              particle_id        vx        vy       vz       px        py  \
12140  968286151951515648 -0.004178  0.004751 -5.12884 -0.45313  0.008006   

            pz  q  nhits  
12

             particle_id        vx        vy       vz        px        py  \
5568  418835864857083904  0.018366 -0.016865  4.19268 -0.476722 -0.181538   

           pz  q  nhits  
5568 -5.25052 -1     10  
             particle_id        vx        vy       vz       px        py  \
8707  711571970939944960  0.003571 -0.012305 -2.64198 -1.19514 -0.487085   

             pz  q  nhits  
8707 -16.141001 -1      9  
             particle_id        vx        vy       vz        px        py  \
9891  801645062998982656 -0.010398 -0.004964  1.15111 -0.514909 -0.164243   

           pz  q  nhits  
9891 -5.61428 -1     13  
              particle_id        vx        vy       vz        px        py  \
12230  968291374631747584 -0.004178  0.004751 -5.12884 -0.165111 -0.033866   

            pz  q  nhits  
12230 -1.65643 -1     15  
             particle_id        vx        vy       vz        px        py  \
8572  707069333385248768 -0.037319 -0.031614 -3.26563 -0.277914 -0.164704   

           

             particle_id        vx        vy       vz        px        py  \
9540  774621197492027392  0.003627  0.003423  5.84297 -0.188628 -0.192407   

           pz  q  nhits  
9540 -4.92988  1     12  
             particle_id        vx        vy       vz        px        py  \
9363  747600836678385664 -0.018073  0.015695  0.35496 -0.123624 -0.143091   

           pz  q  nhits  
9363 -5.03227  1     12  
              particle_id        vx       vy       vz        px       py  \
11471  914233885452140544  0.006451  0.00558 -11.5839 -0.259061 -0.25543   

            pz  q  nhits  
11471 -9.09343  1     10  
              particle_id        vx        vy       vz      px        py  \
11785  941263179797757952  0.000573  0.010101  6.18273 -0.1787 -0.111295   

            pz  q  nhits  
11785 -3.96134 -1     15  
              particle_id        vx        vy       vz        px        py  \
10237  837670630202540032  0.001079  0.013532 -11.1873 -0.281814 -0.253841   

            pz 

              particle_id       vx         vy          vz        px        py  \
11175  873731656656093184 -8.88429 -19.637699 -498.630005 -0.492666 -0.650734   

           pz  q  nhits  
11175 -20.764 -1     12  
             particle_id        vx        vy       vz        px        py  \
9940  801652003666132992 -0.010398 -0.004964  1.15111 -0.147421 -0.183862   

           pz  q  nhits  
9940 -5.09076 -1     12  
             particle_id        vx        vy       vz        px        py  \
3756  252202197608038400 -0.002423  0.001814  6.01673 -0.067345 -0.188417   

           pz  q  nhits  
3756 -3.08195  1     15  
            particle_id        vx        vy        vz        px     py  \
1090  63058434961965056  0.022729  0.005676 -0.535732 -0.890803 -1.348   

             pz  q  nhits  
1090 -29.009199 -1     15  
             particle_id        vx        vy       vz        px        py  \
3858  270217901787578368 -0.011215 -0.030577 -8.24052 -0.253373 -0.380558   

           

              particle_id        vx        vy       vz        px        py  \
11934  954768481121730560  0.014798 -0.019703 -11.7615 -0.046421 -0.263008   

            pz  q  nhits  
11934 -2.74936  1     10  
              particle_id        vx        vy       vz        px        py  \
10122  819656850168348672 -0.013211 -0.005251  3.45567 -0.017907 -0.153218   

            pz  q  nhits  
10122 -3.78807  1     10  
             particle_id        vx        vy       vz        px        py  \
9737  797142700322193408  0.007113  0.001641 -1.68351 -0.021613 -0.332372   

           pz  q  nhits  
9737 -5.41762  1     16  
             particle_id        vx        vy       vz        px        py  \
3673  238694353663426560 -0.012553  0.009759  5.53005 -0.096757 -0.621008   

           pz  q  nhits  
3673 -11.5164  1     14  
              particle_id        vx        vy       vz        px        py  \
10490  851186239448023040  0.002644  0.011603  9.48441 -0.142517 -0.704979   

       

...

              particle_id        vx        vy       vz        px      py  \
10027  810651506339479552  0.003366  0.002173  10.2654  0.748758  0.0947   

            pz  q  nhits  
10027 -10.0966 -1     14  
             particle_id        vx        vy       vz        px        py  \
4050  292749918697684992  0.009175  0.008979  11.2888  0.387217  0.099004   

           pz  q  nhits  
4050 -4.03489  1     11  
             particle_id       vx        vy       vz        px        py  \
7025  585474205030547456 -0.01082 -0.003106  6.37972  0.483788  0.135517   

           pz  q  nhits  
7025 -5.50801  1     10  
              particle_id         vx       vy          vz        px        py  \
10546  851193592432033792  48.710098  7.61727 -501.286011  0.928687  0.115746   

            pz  q  nhits  
10546 -9.93509  1     11  
             particle_id        vx        vy       vz        px        py  \
2429  162130548658012160  0.018001 -0.011872  3.42211  0.507798  0.188375   

         

             particle_id        vx        vy       vz        px       py  \
7786  662029420101369856 -0.000737 -0.006179 -10.0669  0.804261  0.45839   

           pz  q  nhits  
7786 -10.9665  1     14  
             particle_id        vx        vy       vz        px        py  \
4118  297238400040173568 -0.000486 -0.015051  5.75865  0.410131  0.246767   

           pz  q  nhits  
4118 -5.71329  1     15  
             particle_id        vx        vy       vz        px        py  \
9113  738591163522482176  0.000326 -0.044719 -9.97816  0.324172  0.207157   

           pz  q  nhits  
9113 -3.38314  1     13  
           particle_id        vx        vy         vz        px        py  \
767  27039739706081280 -0.387946  0.284646 -60.076099  0.144503  0.111737   

          pz  q  nhits  
767 -1.95208  1     11  
             particle_id       vx        vy       vz       px        py  \
9474  752103336794128384 -0.00901 -0.017719 -8.11248  0.76547  0.300854   

           pz  q  nhits  

             particle_id        vx        vy        vz       px        py  \
1782  117101424331980800  0.000487 -0.009182  0.208295  0.68313  0.673737   

             pz  q  nhits  
1782 -20.264999 -1     13  
             particle_id        vx        vy       vz        px        py  \
4362  306247798318170112 -0.006925 -0.005512  9.08789  0.381912  0.307159   

           pz  q  nhits  
4362 -8.57674 -1     13  
              particle_id        vx        vy       vz        px        py  \
11320  878220962632302592  0.013922  0.004848  1.40247  0.116042  0.138331   

           pz  q  nhits  
11320 -2.7494  1     12  
             particle_id        vx        vy       vz        px       py  \
9497  761108818061950976 -0.003887  0.024822 -15.7559  0.392026  0.38464   

           pz  q  nhits  
9497 -12.6509 -1     11  
             particle_id       vx      vy          vz        px       py  \
6525  517930174944116736  10.3537  5.0666 -375.027008  0.131576  0.10194   

           pz  

             particle_id       vx        vy       vz       px       py  \
9478  752104298866802688 -0.00901 -0.017719 -8.11248  0.31272  1.26746   

             pz  q  nhits  
9478 -35.579899  1     11  
             particle_id        vx        vy      vz        px        py  \
2621  180145359484354560 -0.010003 -0.022239  11.077  0.270509  0.685391   

             pz  q  nhits  
2621 -18.078501 -1     12  
             particle_id        vx        vy        vz        px        py  \
1651  112595488242401280  0.001906 -0.000947 -0.098618  0.134885  0.434474   

           pz  q  nhits  
1651 -5.54762  1     13  
             particle_id       vx        vy       vz        px       py  \
8955  734098215414005760 -0.00917 -0.004763 -2.64611  0.435253  1.06291   

           pz  q  nhits  
8955 -10.2541  1     11  
             particle_id        vx       vy       vz        px       py  \
8121  675540631300341760  0.009943  0.01849 -1.58384  0.079324  0.35984   

          pz  q  nhits 

KeyboardInterrupt: 

In [ ]:
truth['arclength'] = truth.particle_id.map(d2)
truth['p_norm'] = truth.particle_id.map(d3)
df = truth[truth.particle_id != 0]

In [ ]:
particle_dict = particles.set_index('particle_id')[['px','py','pz','q']].to_dict()
#Add charge q
df['q'] = df.particle_id.map(particle_dict['q'])
df[:10]

In [ ]:
x = list(df.arclength)
y = list(df.weight)
plt.scatter(x, y)
plt.title('Arclength vs. Weight')
plt.xlabel('x')
plt.ylabel('y')
plt.ylim((0.0, 0.000175)) 
plt.show()

In [ ]:
df.arclength.corr(df.weight)

In [ ]:
for particle_id,_ in particle_dict['px'].items():
    print(particle_id,_)
    break